## Analisis Equifax

In [39]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
dfPersonas = pd.read_csv('Datos/PERSONAS.csv',delimiter=';')
dfOperaciones = pd.read_csv('Datos/OPERACIONES.csv',delimiter=';')
HHold = pd.read_csv('Datos/HOUSEHOLD.csv',delimiter=';')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Obs: las variables, flag_tarjeta y nivel_bancarizacion tiene varianza 0 -> las eliminamos, tener presente que esta bs es foto, al 2023-08.

In [3]:
dfPersonas = dfPersonas.drop(['nivel_bancarizacion','flag_tarjeta'],axis=1)
dfOperaciones['PERIODO_INFO'] = pd.to_datetime(dfOperaciones['PERIODO_INFO'],format='%Y-%m')
dfOperaciones['FECHA_OTORGAMIENTO'] = pd.to_datetime(dfOperaciones['FECHA_OTORGAMIENTO'],format='%Y-%m')
def convertir_fecha(fecha):
    if len(fecha) == 8:  
        return pd.to_datetime(fecha, format='%Y%m%d')
    elif len(fecha) == 6: 
        return pd.to_datetime(fecha, format='%Y%m')  # asumiendo el primer día del mes
    else:
        return np.nan  
HHold['periodo'] = HHold['periodo'].astype(str).apply(convertir_fecha)

In [22]:
def createdummies(df,varname):
    dummy = pd.get_dummies(df[varname], prefix=varname)
    df = df.drop(varname,axis=1)
    df = pd.concat([df,dummy],axis=1)
    return df

In [49]:
# cate_cols = ['sexo','generaciones','situacion_laboral','provincia','region_arg','flag_prestamo'] 
# dfPersonas[cate_cols] = dfPersonas[cate_cols].astype('category')

print(dfPersonas['pers_id'].size)
#dfPersonas.isnull().sum()

400000


In [43]:
#dfPersonas = dfPersonas.rename(columns={'generaciones':'gen'})
#dfPersonas['sexo'] = dfPersonas['sexo'].replace({'M':1,'F':0})
#dfPersonas = pd.get_dummies(dfPersonas,columns=['sexo'],drop_first=True)
#dfPersonas = createdummies(dfPersonas,'gen')
dfPersonas = createdummies(dfPersonas,'flag_prestamo')

In [38]:
dfPersonas[dfPersonas['flag_prestamo'].isna() == True]

,pers_id,sexo,region_arg,provincia,flag_prestamo,ip20_continuo,gen_18 a 25 años,gen_26 a 41 años,gen_42 a 57 años,gen_> 58 años,situacion_laboral_Autonomo/Monotributista,situacion_laboral_Jubilado,situacion_laboral_Relacion dependencia,situacion_laboral_Sin situacion declarada
0,62b1aa73ac1deaf2e7e978c27d3bdac638c7e6761b622d...,0,CUYO,LA RIOJA,NaN,66116.734375,0,0,0,1,0,1,0,0
1,5a7ab740fa89f46eae34b6bc8e26e487c74cb71e2a1aa0...,0,CUYO,LA RIOJA,NaN,129538.828125,0,0,0,1,0,1,0,0
2,bd0cdae73ef2470eec5d71496c02cea577011741829c53...,0,CUYO,LA RIOJA,NaN,110268.507812,0,0,0,1,0,1,0,0
3,8fb4f93c41093fd45d28a7ab84fa53e3083e8d3d18850c...,0,CUYO,LA RIOJA,NaN,42860.722656,0,1,0,0,1,0,0,0
5,89d87537b05263f64aac7f214b1ff4e4ddabbb0b57c879...,0,CUYO,LA RIOJA,NaN,77912.820312,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,e8a1a316fdcfb7ba97d85f97b7dc09da4bae9ddb049234...,1,NOA,SANTIAGO DEL ESTERO,NaN,962654.625000,0,0,1,0,1,0,0,0
399996,ae42da58d14d82f3d03870fdd3a97e5c311ffe6265dc16...,1,NOA,SANTIAGO DEL ESTERO,NaN,52564.500000,0,0,1,0,0,0,0,1
399997,c6800ded723860fb458c7c2fa9d36d7fc86c973934cf7f...,1,NOA,SANTIAGO DEL ESTERO,NaN,42927.792969,0,0,1,0,0,0,0,1
399998,895ed85139606c4be50702f99fff2896a954262ab9f2d3...,1,NOA,SANTIAGO DEL ESTERO,NaN,28445.998047,0,0,1,0,0,0,0,1


In [42]:
df = dfPersonas.dropna(subset=['flag_prestamo','ip20_continuo'])
df['situacion_laboral_Autonomo/Monotributista']
X = df[['sexo','ip20_continuo','gen_18 a 25 años','gen_26 a 41 años','gen_42 a 57 años',
                'gen_> 58 años','situacion_laboral_Autonomo/Monotributista',
                'situacion_laboral_Jubilado','situacion_laboral_Relacion dependencia',
                'situacion_laboral_Sin situacion declarada']]
y = df['flag_prestamo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Mostrar el informe de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0